# Importamos funciones

In [87]:
import pandas as pd
import re
import src.cleaning as fs

# Cargamos csv para su análisis

In [88]:
df = pd.read_csv("input/houses_Madrid.csv", encoding= "utf-8")

## Hacemos una primera exploración de los datos

In [89]:
df.head()

,Unnamed: 0,id,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,...,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
0,0,21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,...,D,False,NaN,NaN,NaN,NaN,False,True,False,False
1,1,21741,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70.0,NaN,3,1.0,NaN,NaN,...,en trámite,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,21740,"Piso en venta en calle del Talco, 68","San Andrés, Madrid",94.0,54.0,2,2.0,NaN,NaN,...,no indicado,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,21739,Piso en venta en calle Pedro Jiménez,"San Andrés, Madrid",64.0,NaN,2,1.0,NaN,NaN,...,en trámite,False,NaN,NaN,NaN,NaN,False,False,True,False
4,4,21738,Piso en venta en carretera de Villaverde a Val...,"Los Rosales, Madrid",108.0,90.0,2,2.0,NaN,NaN,...,en trámite,True,NaN,NaN,True,0.0,True,True,True,True


In [90]:
df.house_type_id.iloc[39:45]

39     HouseType 1: Pisos
40    HouseType 4: Dúplex
41                    NaN
42     HouseType 1: Pisos
43     HouseType 1: Pisos
44     HouseType 1: Pisos
Name: house_type_id, dtype: object

In [91]:
df.columns

Index(['Unnamed: 0', 'id', 'title', 'subtitle', 'sq_mt_built', 'sq_mt_useful',
       'n_rooms', 'n_bathrooms', 'n_floors', 'sq_mt_allotment', 'latitude',
       'longitude', 'raw_address', 'is_exact_address_hidden', 'street_name',
       'street_number', 'portal', 'floor', 'is_floor_under', 'door',
       'neighborhood_id', 'operation', 'rent_price', 'rent_price_by_area',
       'is_rent_price_known', 'buy_price', 'buy_price_by_area',
       'is_buy_price_known', 'house_type_id', 'is_renewal_needed',
       'is_new_development', 'built_year', 'has_central_heating',
       'has_individual_heating', 'are_pets_allowed', 'has_ac',
       'has_fitted_wardrobes', 'has_lift', 'is_exterior', 'has_garden',
       'has_pool', 'has_terrace', 'has_balcony', 'has_storage_room',
       'is_furnished', 'is_kitchen_equipped', 'is_accessible',
       'has_green_zones', 'energy_certificate', 'has_parking',
       'has_private_parking', 'has_public_parking',
       'is_parking_included_in_price', 'parki

In [92]:
df.buy_price.unique()

array([ 85000, 129900, 144247, ..., 215750, 299800, 388000])

In [93]:
df.iloc[1]

Unnamed: 0                                                                      1
id                                                                          21741
title                            Piso en venta en calle de la del Manojo de Rosas
subtitle                                                      Los Ángeles, Madrid
sq_mt_built                                                                    70
sq_mt_useful                                                                  NaN
n_rooms                                                                         3
n_bathrooms                                                                     1
n_floors                                                                      NaN
sq_mt_allotment                                                               NaN
latitude                                                                      NaN
longitude                                                                     NaN
raw_address     

### Comprobamos que no hayan filas duplicadas

In [94]:
df.duplicated().sum()

0

In [95]:
df.shape

(21742, 58)

In [96]:
df["subtitle"] = df.subtitle.apply(lambda x: x.split(",")[0])

In [97]:
df.subtitle.value_counts()

Chamartín                           851
Moncloa                             771
Chamberí                            583
Centro                              552
Hortaleza                           503
                                   ... 
Barajas                               7
Campo de las Naciones-Corralejos      6
Casco Histórico de Barajas            5
Timón                                 4
Cuatro Vientos                        2
Name: subtitle, Length: 146, dtype: int64

In [98]:
df.house_type_id.unique()

array(['HouseType 1: Pisos', 'HouseType 4: Dúplex', nan,
       'HouseType 5: Áticos', 'HouseType 2: Casa o chalet'], dtype=object)

In [99]:
df.is_new_development.unique()

array([False, nan, True], dtype=object)

## Seleccionamos las columnas que nos interesan para proceder a la limpieza de los datos

In [100]:
df2 = df[["subtitle", "sq_mt_built", "n_rooms", "n_bathrooms", "neighborhood_id", "buy_price", "buy_price_by_area", "is_new_development", "house_type_id", "has_garden", "has_pool","has_parking"]]

In [101]:
df2.head(10)

,subtitle,sq_mt_built,n_rooms,n_bathrooms,neighborhood_id,buy_price,buy_price_by_area,is_new_development,house_type_id,has_garden,has_pool,has_parking
0,San Cristóbal,64.0,2,1.0,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,85000,1328,False,HouseType 1: Pisos,NaN,NaN,False
1,Los Ángeles,70.0,3,1.0,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,129900,1856,False,HouseType 1: Pisos,NaN,NaN,False
2,San Andrés,94.0,2,2.0,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,144247,1535,False,HouseType 1: Pisos,NaN,NaN,False
3,San Andrés,64.0,2,1.0,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,109900,1717,False,HouseType 1: Pisos,NaN,NaN,False
4,Los Rosales,108.0,2,2.0,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,260000,2407,False,HouseType 1: Pisos,NaN,True,True
5,San Andrés,126.0,4,2.0,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,195000,1548,False,HouseType 1: Pisos,NaN,NaN,True
6,San Andrés,120.0,5,2.0,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,190000,1583,False,HouseType 1: Pisos,NaN,True,True
7,Villaverde,125.0,3,2.0,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,198500,1588,False,HouseType 1: Pisos,NaN,NaN,False
8,Villaverde,84.0,3,2.0,Neighborhood 131: Butarque (2197.8 €/m2) - Dis...,212000,2524,False,HouseType 1: Pisos,NaN,True,True
9,Los Rosales,85.0,2,1.0,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,131400,1546,False,HouseType 1: Pisos,NaN,NaN,False


In [102]:
df2.isnull().sum()

subtitle                  0
sq_mt_built             126
n_rooms                   0
n_bathrooms              16
neighborhood_id           0
buy_price                 0
buy_price_by_area         0
is_new_development      992
house_type_id           391
has_garden            20186
has_pool              16571
has_parking               0
dtype: int64

In [103]:
df2.shape

(21742, 12)

### Eliminamos las filas con valores nulos y rellenamos las que nos interesa conservar

In [104]:
df2 = df2.dropna(axis=0, subset=["sq_mt_built", "is_new_development", "house_type_id", "n_bathrooms"])

In [105]:
df2.shape

(20238, 12)

In [106]:
df2["has_garden"] = df2.has_garden.fillna("False")

In [107]:
df2["has_pool"] = df2.has_pool.fillna("False")

In [108]:
df2.isnull().sum()

subtitle              0
sq_mt_built           0
n_rooms               0
n_bathrooms           0
neighborhood_id       0
buy_price             0
buy_price_by_area     0
is_new_development    0
house_type_id         0
has_garden            0
has_pool              0
has_parking           0
dtype: int64

## Realizamos la limpieza de los datos aplicando funciones para ellos

In [109]:
df2["numero_barrio"] = df.neighborhood_id.apply(fs.neighborhood)

In [110]:
df2["barrio"] = df.neighborhood_id.apply(fs.barrio)

In [111]:
df2["tipo"] = df2.house_type_id.apply(fs.house_type)

In [112]:
df2.neighborhood_id.iloc[8]

'Neighborhood 131: Butarque (2197.8 €/m2) - District 21: Villaverde'

In [113]:
df2["eurosm2"] = df.neighborhood_id.apply(fs.euro_m2)

In [114]:
df2["numero_distrito"] = df2.neighborhood_id.apply(fs.n_distrito)

In [115]:
df2["distrito"] = df2.neighborhood_id.apply(lambda x: x.split(":")[2].strip())

#### Aqui relleno las columnas buy_price_by_area y eurosm2 con un valor orientativo de 1500 ya que no quiero perder unos 200 pisos simplemente por no tener el valor promedio del m2 construido.

In [116]:
df2["buy_price_by_area"] = df2.buy_price_by_area.apply(lambda x: 1500 if "Neighborhood" in str(x) else x)

In [117]:
df2["eurosm2"] = df2.eurosm2.apply(lambda x: 1500 if "Neighborhood" in str(x) else x)

In [118]:
df2.head(10)

,subtitle,sq_mt_built,n_rooms,n_bathrooms,neighborhood_id,buy_price,buy_price_by_area,is_new_development,house_type_id,has_garden,has_pool,has_parking,numero_barrio,barrio,tipo,eurosm2,numero_distrito,distrito
0,San Cristóbal,64.0,2,1.0,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,85000,1328,False,HouseType 1: Pisos,False,False,False,Neighborhood 135,San Cristóbal,Pisos,1308.89,District 21,Villaverde
1,Los Ángeles,70.0,3,1.0,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,129900,1856,False,HouseType 1: Pisos,False,False,False,Neighborhood 132,Los Ángeles,Pisos,1796.68,District 21,Villaverde
2,San Andrés,94.0,2,2.0,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,144247,1535,False,HouseType 1: Pisos,False,False,False,Neighborhood 134,San Andrés,Pisos,1617.18,District 21,Villaverde
3,San Andrés,64.0,2,1.0,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,109900,1717,False,HouseType 1: Pisos,False,False,False,Neighborhood 134,San Andrés,Pisos,1617.18,District 21,Villaverde
4,Los Rosales,108.0,2,2.0,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,260000,2407,False,HouseType 1: Pisos,False,True,True,Neighborhood 133,Los Rosales,Pisos,1827.79,District 21,Villaverde
5,San Andrés,126.0,4,2.0,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,195000,1548,False,HouseType 1: Pisos,False,False,True,Neighborhood 134,San Andrés,Pisos,1617.18,District 21,Villaverde
6,San Andrés,120.0,5,2.0,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,190000,1583,False,HouseType 1: Pisos,False,True,True,Neighborhood 134,San Andrés,Pisos,1617.18,District 21,Villaverde
7,Villaverde,125.0,3,2.0,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,198500,1588,False,HouseType 1: Pisos,False,False,False,Neighborhood 134,San Andrés,Pisos,1617.18,District 21,Villaverde
8,Villaverde,84.0,3,2.0,Neighborhood 131: Butarque (2197.8 €/m2) - Dis...,212000,2524,False,HouseType 1: Pisos,False,True,True,Neighborhood 131,Butarque,Pisos,2197.8,District 21,Villaverde
9,Los Rosales,85.0,2,1.0,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,131400,1546,False,HouseType 1: Pisos,False,False,False,Neighborhood 133,Los Rosales,Pisos,1827.79,District 21,Villaverde


## Generamos el dataframe definitivo

In [119]:
df_clean = df2[["distrito", "numero_distrito", "barrio",  "numero_barrio", "tipo", "is_new_development", "sq_mt_built", "n_rooms", "n_bathrooms", "has_garden", "has_pool","has_parking", "buy_price", "buy_price_by_area", "eurosm2"]]

In [120]:
df_clean.head(10)

,distrito,numero_distrito,barrio,numero_barrio,tipo,is_new_development,sq_mt_built,n_rooms,n_bathrooms,has_garden,has_pool,has_parking,buy_price,buy_price_by_area,eurosm2
0,Villaverde,District 21,San Cristóbal,Neighborhood 135,Pisos,False,64.0,2,1.0,False,False,False,85000,1328,1308.89
1,Villaverde,District 21,Los Ángeles,Neighborhood 132,Pisos,False,70.0,3,1.0,False,False,False,129900,1856,1796.68
2,Villaverde,District 21,San Andrés,Neighborhood 134,Pisos,False,94.0,2,2.0,False,False,False,144247,1535,1617.18
3,Villaverde,District 21,San Andrés,Neighborhood 134,Pisos,False,64.0,2,1.0,False,False,False,109900,1717,1617.18
4,Villaverde,District 21,Los Rosales,Neighborhood 133,Pisos,False,108.0,2,2.0,False,True,True,260000,2407,1827.79
5,Villaverde,District 21,San Andrés,Neighborhood 134,Pisos,False,126.0,4,2.0,False,False,True,195000,1548,1617.18
6,Villaverde,District 21,San Andrés,Neighborhood 134,Pisos,False,120.0,5,2.0,False,True,True,190000,1583,1617.18
7,Villaverde,District 21,San Andrés,Neighborhood 134,Pisos,False,125.0,3,2.0,False,False,False,198500,1588,1617.18
8,Villaverde,District 21,Butarque,Neighborhood 131,Pisos,False,84.0,3,2.0,False,True,True,212000,2524,2197.8
9,Villaverde,District 21,Los Rosales,Neighborhood 133,Pisos,False,85.0,2,1.0,False,False,False,131400,1546,1827.79


In [121]:
df_clean.shape

(20238, 15)

### Renombramos las columnas para una mejor comprensión de los datos

In [122]:
df_clean.columns = ["distrito", "numero_distrito", "barrio",  "numero_barrio", "tipo", "nueva_construccion", "metros2_const", "n_habitaciones", "n_banos", "jardin", "piscina","parking", "precio_venta", "eurosm2_distrito" ,"eurosm2_barrio"]

In [123]:
df_clean.head(10)

,distrito,numero_distrito,barrio,numero_barrio,tipo,nueva_construccion,metros2_const,n_habitaciones,n_banos,jardin,piscina,parking,precio_venta,eurosm2_distrito,eurosm2_barrio
0,Villaverde,District 21,San Cristóbal,Neighborhood 135,Pisos,False,64.0,2,1.0,False,False,False,85000,1328,1308.89
1,Villaverde,District 21,Los Ángeles,Neighborhood 132,Pisos,False,70.0,3,1.0,False,False,False,129900,1856,1796.68
2,Villaverde,District 21,San Andrés,Neighborhood 134,Pisos,False,94.0,2,2.0,False,False,False,144247,1535,1617.18
3,Villaverde,District 21,San Andrés,Neighborhood 134,Pisos,False,64.0,2,1.0,False,False,False,109900,1717,1617.18
4,Villaverde,District 21,Los Rosales,Neighborhood 133,Pisos,False,108.0,2,2.0,False,True,True,260000,2407,1827.79
5,Villaverde,District 21,San Andrés,Neighborhood 134,Pisos,False,126.0,4,2.0,False,False,True,195000,1548,1617.18
6,Villaverde,District 21,San Andrés,Neighborhood 134,Pisos,False,120.0,5,2.0,False,True,True,190000,1583,1617.18
7,Villaverde,District 21,San Andrés,Neighborhood 134,Pisos,False,125.0,3,2.0,False,False,False,198500,1588,1617.18
8,Villaverde,District 21,Butarque,Neighborhood 131,Pisos,False,84.0,3,2.0,False,True,True,212000,2524,2197.8
9,Villaverde,District 21,Los Rosales,Neighborhood 133,Pisos,False,85.0,2,1.0,False,False,False,131400,1546,1827.79


In [124]:
df_clean.shape

(20238, 15)

## Exportamos el csv definitivo para su posterior análisis

In [125]:
df_clean.to_csv("output/clean_csv.csv", index = False)